In [1]:
from helpers import *

import tqdm
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch

In [2]:
def sample(x, y, bs):
    idx = np.random.choice(x.size(0), bs)
    return x[idx].cuda(), y[idx].cuda()
    
def train_model(tr_feat, tr_lbl, te_feat, te_lbl, k=5, bs=2000, nbatch=500):
    """
    """
    vis_dim = tr_feat.size(1)
    n_class = tr_lbl.max() + 1
    
    model = nn.Linear(vis_dim, n_class, bias=False).cuda()
    crit  = nn.CrossEntropyLoss()
    opt   = torch.optim.Adam(model.parameters())
    accs,losses  = [],[]
    
    for i in tqdm.tqdm(range(1,nbatch+1)):
        x,y  = sample(tr_feat, tr_lbl, bs)
        losses.append(train(x, y, model, crit, opt))#, base_idx)
        accs.append(validate(te_feat, te_lbl, model))
        
    return model, losses, accs

def train(tr, lbl, model, crit, opt, base_idx=None):
    """
    """
    opt.zero_grad()
    out  = model(tr)
    loss = crit(out, lbl)
    loss.backward()
    opt.step()
    return loss.item()

def validate(feat, lbl, model, k=5):
    """
    """
    out = model(feat).topk(k)[1]
    return (out == lbl.unsqueeze(1)).float().sum(1).mean().item()

def scores_cons(v_te, s_te, s_tr, w, k=5):
    a,b = torch.nn.Softmax(-1)(w(v_te)).topk(k)
    out = F.normalize((a.unsqueeze(-1) * s_tr[b]).mean(1))
    return  out.mm(s_te.t())

# Training

In [3]:
x_tr, y_tr, s_tr = load_train_set(norm_sem=True)
tenodes = pickle.load(open(train_split, "rb"))

x_val, y_val = load_visuals(tenodes, test_feature_path)
model, losses, accs = train_model(x_tr, y_tr, x_val.cuda(), y_val.cuda())

993it [00:51, 19.41it/s]
1000it [00:17, 58.01it/s]
100%|██████████| 500/500 [00:22<00:00, 22.66it/s]


# Standard ZSL

In [4]:
x_te,y_te,s_te = load_test_set(generalized=False, norm_sem=True)

500it [00:06, 79.14it/s] 


In [5]:
out=scores_cons(x_te.cuda(), s_te.cuda(), s_tr.cuda(), model)
test_accs = topk(out, y_te.cuda())
pp(test_accs)

 10.65 | 15.83 | 19.69 | 22.75 | 25.10 


# Generalized ZSL

In [6]:
x_te,y_te,s_te=load_test_set(generalized=True, norm_sem=True)

1493it [00:00, 1857.07it/s]


In [7]:
msk_test  = y_te<500
out=scores_cons(x_te.cuda(), s_te.cuda(), s_tr.cuda(), model)
test_accs = topk(out[msk_test], y_te[msk_test].cuda())
pp(test_accs)

 0.06 | 4.35 | 6.92 | 9.00 | 10.91 


In [8]:
msk_train = y_te>500
out=scores_cons(x_te.cuda(), s_te.cuda(), s_tr.cuda(), model)
train_accs = topk(out[msk_train], y_te[msk_train].cuda())
pp(train_accs)

 73.48 | 80.35 | 82.57 | 83.86 | 84.79 


In [9]:
pp(h_scores(test_accs, train_accs))

 0.12 | 8.25 | 12.77 | 16.26 | 19.34 
